# beomi/KcELECTRA-base

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10828426691359273100
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2254123828
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6701334500120241282
physical_device_desc: "device: 0, name: GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [3]:
# %run KcELECTRA-base.version.py
# !pip freeze

In [4]:
import re
import torch
import pandas as pd

# deivce 선택
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("devcie:", device)

devcie: cpu


In [5]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\sentence_data.xlsx")

# 'Sentence' 칼럼의 값을 문자열로 변환하여 리스트로 저장
str_data = df['Sentence'].astype(str).tolist()

# 'Sentence' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(str_data)

print(df.shape)
df.head()

(134197, 2)


,Sentence,label
0,좌배 까는건 ㅇㅂ,1
1,집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
2,개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
3,세탁이라고 봐도 된다,0
4,애새끼가 초딩도 아니고 ㅋㅋㅋㅋ,1


In [6]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [7]:
import sys
sys.path.append(r'C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\Morphological_Analyzer')

In [8]:
from kiwi import *

kiwi_pre_train = df.sample(frac=0.8, random_state=42)                     # train(80%), test(20%) 셋 구분 
kiwi_pre_test = df.drop(kiwi_pre_train.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(kiwi_pre_train)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(kiwi_pre_test)))

# 중복 데이터 제거
kiwi_pre_train.drop_duplicates(subset=["Sentence"], inplace=True)
kiwi_pre_test.drop_duplicates(subset=["Sentence"], inplace=True)

# 형태소 분석 적용
train_data = list(kiwi.tokenize(kiwi_pre_train['Sentence'].tolist(), normalize_coda=True))
test_data = list(kiwi.tokenize(kiwi_pre_test['Sentence'].tolist(), normalize_coda=True))

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 107358
중복 제거 전 테스트 데이터셋 : 26839
중복 제거 후 학습 데이터셋 : 103690
중복 제거 후 테스트 데이터셋 : 26600


In [16]:
import pandas as pd

# 리스트 내포를 사용하여 데이터프레임 생성
train_df = pd.DataFrame([[token.form for token in inner_list] for inner_list in train_data])

test_df = pd.DataFrame([[token.form for token in inner_list] for inner_list in test_data])

# 데이터프레임 출력
print(test_df)

         0   1    2     3    4   5     6     7     8     9    ...   247   248  \
0        개소리   야    니     가  빨갱이   를    옹호     하     고   드루킹  ...  None  None   
1      731부대   의   후예     이    라  그렇    ᆫ지    가학     적     이  ...  None  None   
2        대가리   에   필터     없    는  연봉    30     억    강사    vs  ...  None  None   
3          뭐  저런    골     비    ᆫ   ㄴ     이     다     있     냐  ...  None  None   
4          열  사발  들이키    어도   아깝   지     않     음  None  None  ...  None  None   
...      ...  ..  ...   ...  ...  ..   ...   ...   ...   ...  ...   ...   ...   
26595      '   '  헬조선     '    '   '     '     '    김치    ''  ...  None  None   
26596    한의사   이    라    벌이    가  그닥  None  None  None  None  ...  None  None   
26597      지   들    은  #@이름    #  만나     ᆯ     수     있     을  ...  None  None   
26598     현역   갓    나     아    ?  상근     들     은    제발     군  ...  None  None   
26599      '   '    반     일    짓   '     '     '     하    다가  ...  None  None   

        249   250   251   2

In [21]:
from nltk.corpus import stopwords
from nltk.tokenize import  word_tokenize
from konlpy.tag import Okt

for i in train_df:
    for j in i:
        train_df[i][j] = 

개소리


In [14]:

# 형태소 분석이 끝난 데이터프레임에서 각 행의 토큰을 문자열로 결합하여 새로운 칼럼 추가
train_df['merged_sentences'] = train_df.apply(lambda row: ' '.join([str(token) for token in row if token is not None]), axis=1)
test_df['merged_sentences'] = test_df.apply(lambda row: ' '.join([str(token) for token in row if token is not None]), axis=1)


# 'merged_sentences' 칼럼만 추출하여 하나의 데이터프레임으로 합치기
merged_df = pd.DataFrame({'merged_sentences': train_df['merged_sentences'].tolist() + test_df['merged_sentences'].tolist()})

# 'merged_sentences' 칼럼의 값을 하나의 문자열로 결합
all_sentences = ' '.join(merged_df['merged_sentences'].tolist())

# 결과 확인
print(merged_df.head())



                                    merged_sentences
0  세계 경제 는 일본 ' ' 놈' ' ' 이 ' ' 말 아 처먹 ' ' ' 고 한국경...
1                                    기안내나노 ㅋ 기안내나노 ㅋ
2  진영 논리 빼 고 ᆫ 아무것 도 없 는 ' ' 스레기당' ' ' 민주당 이 하 는 ...
3  개인 적 으로 기독교 비 호감 이래 ㅋㅋㅋㅋㅋ 지랄 하 ᆫ다 ㅋㅋㅋㅋㅋ 오랑캐 종교...
4  그래 ~ 북조선 애 들 보다 차라리 저런 애 들 걷 어 주 는 것 이 더 낫 다 ....


In [ ]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenized_train_sentences = tokenizer(
    list(train_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
tokenized_test_sentences = tokenizer(
    list(test_data["Sentence"]),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [ ]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
    def __init__(self, encoding, labels):
        self.encodings = encoding
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [ ]:
# train_set, test_set에 대한 데이터셋을 각각 생성

train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)   # 사전 학습된 모델 찾아오기
model.to(device)

In [ ]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로
    logging_steps=500,                  # 학습 log 기록 단위
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [ ]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = model,                       # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 Trainging Arguments
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [ ]:
type(trainer)

In [ ]:
trainer.train()

In [ ]:
# 평가를 수행하고 custom_ 접두사가 붙은 평가 지표를 출력
results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='custom_')

print(results)

In [ ]:
# 0: curse, 1: non_curse
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    # tokenized_sent.to(deivce)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids=tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    sentence = True
    input_sentence = tokenized_sent["input_ids"]
    if result == 0:
        sentence = True

    elif result == 1:
        sentence = False
    return sentence, input_sentence


sentence_predict("씨발이게 맞아?")

In [ ]:
def found_word(input_sentence, found_bad_word):
    result = input_sentence
    if found_bad_word:
        result = badword_find(result)
    return result

def badword_find(input_sentence):
    result = input_sentence
    badword_df = pd.read_excel(r'C:\Users\user\Desktop\X-filter\word_list.xlsx')
    
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in input_sentence:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                result = result.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                result = result.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        result = "혐오 표현입니다."
    return result

# 테스트
input_sentence = "엄마가 죽었으면 좋겠어"
speak_result = badword_find(input_sentence)
print(speak_result)

In [ ]:
badword_find(input_sentence)

In [ ]:
def speak(sent):
    sentence = sentence_predict(sent)
    if sentence[0] == False:
        return sent
    elif sentence[0] == True:
        return badword_find(sent)

In [ ]:
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           #  제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("아버지가방에들어가신다")

In [ ]:
def final_output():
    input_text = clean_and_repeat_normalize(input())
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        text_output = speak(sentence)
        sentences_list.append(text_output)

    long_test = ' '.join(sentences_list)
    print(long_test)
    return long_test

In [ ]:
final_output()